In [ ]:
import pandas as pd
from autocorrect import Speller

# add our stuff to the path
import sys
import os
import re
sys.path.insert(1, os.path.join(sys.path[0], '..'))

# import our stuff
from importlib import reload
from src import q34

import warnings
warnings.simplefilter(action='ignore')

In [ ]:
reload(q34)

In [ ]:
path = "../data/AQ34CULTURE 20230908.xlsx"

In [ ]:
df = pd.read_excel(path)

In [ ]:
df_codes = pd.read_excel(path, sheet_name=1)
df_codes.head()
code_list = df_codes.iloc[:, 0].values
code_list

In [ ]:
df.columns = ['response']

In [ ]:
df.response = df.response.astype(str)

In [ ]:
spell = Speller()

# get a list of all words that are directly in the code words 
all_words = []
for word in code_list:
    words = re.split(r'\sand\s|[,;()/\r\n\s]+', word)
    for x in words:
        if len(x) > 0:
            all_words.append(x)
            
# add additional words that are not inaccurate 
words = [
    'Salish', 
    'Collectivist', 
    'Konger', 
    'Kong', 
    'Slavic', 
    'Colonizer', 
    'Tagalog', 
    'Syilx', 
    'Panasian', 
    'Malayali', 
    'Jatt', 
    'Metis',
    'Romani',
    'Slovak',
    'Ilocano'
]

for word in all_words + words:

    for x in [word, word.upper(), word.lower()]:
        if x in spell.nlp_data:
            continue
            
        spell.nlp_data[word] = 100
        spell.nlp_data[word.upper()] = 100
        spell.nlp_data[word.lower()] = 100

In [ ]:
clean_dict = {
    'response': [], 
    'cleaned': [],
    #'translated': [],
    #'code': [],
    'exact_match': [],
    'partial_match': [],
    'exact_match_codes': [],
    'partial_match_codes': [],
    'likely_match_codes': []
}

n_rows = df.shape[0]

for idx, row in df.iterrows():

    x = row.response
    response, cleaned, has_exact, has_partial, exact_match_codes, partial_match_codes, likely_match_codes = q34.do_the_things(x, spell, code_list)
    
    clean_dict['response'].append(response)
    clean_dict['cleaned'].append(cleaned)
    #clean_dict['translated'].append(translated)
    #clean_dict['code'].append(response_code)
    clean_dict['exact_match'].append(has_exact)
    clean_dict['partial_match'].append(has_partial)
    clean_dict['exact_match_codes'].append(exact_match_codes)
    clean_dict['partial_match_codes'].append(partial_match_codes)
    clean_dict['likely_match_codes'].append(likely_match_codes)
    
    pct_done = int(round(100*(idx+1)/n_rows))
    print_line = f'{idx+1:07,}/{n_rows:07,}   |' + '-'*(pct_done) + '>' + ' '*(100-pct_done) + '|'
    print(print_line, end = '\r')

In [ ]:
clean_df = pd.DataFrame(clean_dict)

In [ ]:
clean_df

In [ ]:
exact = clean_df.exact_match.sum()
n_rows = clean_df.shape[0]
partial_no_exact = clean_df[~clean_df.exact_match].partial_match.sum()
n_rows_no_exact = clean_df[~clean_df.exact_match].shape[0]
print(f'Exact Matches: {exact:,}/{n_rows:,} ({exact/n_rows:.0%})')
print(f'Partial Matches: {partial_no_exact:,}/{n_rows_no_exact:,} ({partial_no_exact/n_rows_no_exact:.0%})')
print(f'Leftover: {n_rows - exact - partial_no_exact:,}/{n_rows:,} ({(n_rows - exact - partial_no_exact)/n_rows:.0%})')

In [ ]:
with pd.ExcelWriter(path, mode='a', if_sheet_exists='replace') as writer:
    clean_df.to_excel(writer, sheet_name = 'Responses - Cleaned', index=False)